In [ ]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
!pip install transformers>=4.40.0 torch>=2.0.0 accelerate scikit-learn pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F  # ✅ Added for Focal Loss
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split  # ✅ Added for proper split
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import (
    AutoTokenizer,
    BertModel,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from torch.utils.data import Dataset
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [ ]:
class BitLinear(nn.Module):
    """
    1.58-bit Quantized Linear Layer (BitNet)

    Key Features:
    - Weights: Ternary quantization {-1, 0, +1}
    - Activations: 8-bit quantization [-128, 127]
    - Straight-Through Estimator (STE) for gradient flow
    - Lambda warmup for gradual quantization
    """
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        # Initialize weights with Xavier uniform (better for deep networks)
        self.weight = nn.Parameter(torch.empty(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        if bias:
            self.bias = nn.Parameter(torch.zeros(out_features))
        else:
            self.register_parameter('bias', None)

        # Layer normalization before quantization (critical for stability)
        self.layer_norm = nn.LayerNorm(in_features)

        # Lambda for gradual quantization warmup (starts at 0, goes to 1)
        self.register_buffer('lambda_val', torch.tensor(0.0))

    def weight_quant(self, w):
        """
        Quantize weights to ternary values {-1, 0, +1}
        Uses round-to-nearest with scale normalization
        """
        # Calculate scale factor using mean absolute value
        scale = 1.0 / w.abs().mean().clamp_(min=1e-5)
        # Round to nearest integer and clamp to [-1, 0, 1]
        w_quant = (w * scale).round().clamp_(-1, 1) / scale
        return w_quant

    def activation_quant(self, x):
        """
        Quantize activations to 8-bit using absmax quantization
        Maps to [-128, 127] range
        """
        # Find maximum absolute value per sample
        scale = 127.0 / x.abs().max(dim=-1, keepdim=True).values.clamp_(min=1e-5)
        # Quantize and dequantize
        x_quant = (x * scale).round().clamp_(-128, 127) / scale
        return x_quant

    def forward(self, x):
        # Apply layer normalization first
        x_norm = self.layer_norm(x)

        # Get current lambda value (controls quantization strength)
        lambda_val = self.lambda_val.item()

        if self.training:
            # During training: gradual quantization with lambda warmup
            x_quant_full = self.activation_quant(x_norm)
            w_quant_full = self.weight_quant(self.weight)

            # Linear interpolation between full precision and quantized
            # Lambda = 0: full precision, Lambda = 1: full quantization
            x_mixed = x_norm * (1 - lambda_val) + x_quant_full * lambda_val
            w_mixed = self.weight * (1 - lambda_val) + w_quant_full * lambda_val

            # Straight-Through Estimator: forward with quantized, backward with original
            x_final = x_mixed + (x_quant_full - x_mixed).detach()
            w_final = w_mixed + (w_quant_full - w_mixed).detach()
        else:
            # During inference: full quantization (lambda = 1)
            x_final = x_norm + (self.activation_quant(x_norm) - x_norm).detach()
            w_final = self.weight + (self.weight_quant(self.weight) - self.weight).detach()

        # Standard linear transformation
        return F.linear(x_final, w_final, self.bias)

In [ ]:
class BitNetBinaryClassifier(nn.Module):
    """
    BitNet model for binary polarization detection
    Architecture: BERT -> BitLinear Layers -> Classification
    """
    def __init__(self, model_name='bert-base-uncased', num_labels=2, dropout_prob=0.1):
        super().__init__()

        # Load pretrained BERT
        print(f"Loading BERT model: {model_name}")
        self.bert = BertModel.from_pretrained(model_name)
        config = self.bert.config
        self.num_labels = num_labels

        # Optional: Freeze early BERT layers for efficiency
        # Uncomment to freeze first 8 layers (keeps last 4 trainable)
        # for layer in self.bert.encoder.layer[:8]:
        #     for param in layer.parameters():
        #         param.requires_grad = False

        # BitLinear classification head (2 layers for better representation)
        self.dropout = nn.Dropout(dropout_prob)
        self.bit_fc1 = BitLinear(config.hidden_size, config.hidden_size // 2)
        self.activation = nn.GELU()
        self.bit_fc2 = BitLinear(config.hidden_size // 2, num_labels)

        print(f"Model initialized with {sum(p.numel() for p in self.parameters()):,} parameters")

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        # Get BERT embeddings
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # Use [CLS] token representation (pooled output)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # Pass through BitLinear layers
        x = self.bit_fc1(pooled_output)
        x = self.activation(x)
        x = self.dropout(x)
        logits = self.bit_fc2(x)

        # Compute loss if labels provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # Return in HuggingFace format
        return SequenceClassifierOutput(
              loss=loss,
              logits=logits,
              hidden_states=None,
              attentions=None)

In [ ]:
class PolarizationDataset(Dataset):
    """Dataset class for polarization detection"""
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Tokenize with proper truncation
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding=False,  # Handled by DataCollator
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Squeeze to remove batch dimension
        item = {key: encoding[key].squeeze() for key in encoding.keys()}
        item['labels'] = torch.tensor(label, dtype=torch.long)

        return item

In [ ]:
class FocalLoss(nn.Module):
    """
    Focal Loss for handling class imbalance
    Reference: https://arxiv.org/abs/1708.02002

    Better than weighted CE for imbalanced classification because it:
    - Focuses on hard-to-classify examples
    - Down-weights easy examples
    - Reduces false positives
    """
    def __init__(self, alpha=0.65, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss


class BitNetTrainer(Trainer):
    """
    Custom trainer with:
    - Gradual quantization warmup (lambda scheduling)
    - Option for Weighted CE or Focal Loss for class imbalance
    """
    def __init__(self, warmup_steps=1000, class_weight=None, use_focal_loss=False, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.warmup_steps = warmup_steps
        self.use_focal_loss = use_focal_loss

        # Handle class weights
        if class_weight is not None:
            self.class_weight = class_weight.to(self.args.device)
        else:
            self.class_weight = None

        print(f"Lambda warmup enabled: 0 -> 1 over {warmup_steps} steps")

        # Initialize loss function
        if self.use_focal_loss:
            self.focal_loss = FocalLoss(alpha=0.65, gamma=2.0)
            print(f"Using Focal Loss (alpha=0.65, gamma=2.0)")
        elif self.class_weight is not None:
            print(f"Using Weighted CE Loss with weights: {self.class_weight}")
        else:
            print(f"Using standard Cross-Entropy Loss")

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Calculate lambda based on current training step
        current_step = self.state.global_step
        lambda_val = min(1.0, current_step / self.warmup_steps)

        # Set lambda for all BitLinear layers
        for module in model.modules():
            if hasattr(module, 'lambda_val'):
                module.lambda_val.fill_(lambda_val)

        # Get labels and perform forward pass
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Compute loss based on selected method
        if self.use_focal_loss:
            loss = self.focal_loss(logits, labels)
        elif self.class_weight is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weight)
            loss = loss_fct(logits, labels)
        else:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
def compute_metrics(eval_pred):
    """Compute metrics for binary classification"""
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)

    return {
        'f1_macro': f1_score(labels, preds, average='macro'),
        'f1_binary': f1_score(labels, preds, average='binary'),
        'accuracy': accuracy_score(labels, preds)
    }

In [ ]:
def train_polarization_detector():
    """
    OPTIMIZED training function for maximizing F1 Macro

    Key optimizations:
    - Proper 80/20 train/val split with stratification
    - Focal Loss for better class imbalance handling
    - Increased dropout (0.2) and weight decay (0.02)
    - 10 epochs with early stopping
    - Optimized learning rate (2e-5)
    - FP16 mixed precision for faster training
    """
    print("="*60)
    print("SUBTASK 1: BINARY POLARIZATION DETECTION")
    print("1-Bit LLM with BitLinear Quantization - OPTIMIZED")
    print("="*60)

    # Load and split data
    print("\nLoading data...")
    train_full = pd.read_csv('train_eng.csv')

    # Proper 80/20 split with stratification
    train, val = train_test_split(
        train_full,
        test_size=0.2,
        stratify=train_full['polarization'],
        random_state=42
    )

    print(f"Training samples: {len(train)}")
    print(f"Validation samples: {len(val)}")
    print(f"\nTraining label distribution:")
    print(train['polarization'].value_counts())
    print(f"\nValidation label distribution:")
    print(val['polarization'].value_counts())

    # Initialize tokenizer
    model_name = 'bert-base-uncased'
    print(f"\nInitializing tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Create datasets
    print("Creating datasets...")
    train_dataset = PolarizationDataset(
        train['text'].tolist(),
        train['polarization'].tolist(),
        tokenizer,
        max_length=128
    )

    val_dataset = PolarizationDataset(
        val['text'].tolist(),
        val['polarization'].tolist(),
        tokenizer,
        max_length=128
    )

    # Initialize model with optimized dropout
    print("\nInitializing BitNet model...")
    model = BitNetBinaryClassifier(
        model_name=model_name,
        num_labels=2,
        dropout_prob=0.2  # ✅ Increased from 0.1 for better generalization
    )

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")

    # Calculate class weights
    class_weights = torch.tensor([0.82, 1.30], dtype=torch.float32)
    print(f"Class weights: {class_weights}")

    # OPTIMIZED Training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=10,  # ✅ Increased from 7 for better convergence
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        warmup_steps=500,
        learning_rate=2e-5,  # ✅ Optimized learning rate
        weight_decay=0.02,  # ✅ Increased regularization
        logging_dir='./logs',
        logging_steps=50,
        eval_strategy='steps',
        eval_steps=100,
        save_strategy='steps',
        save_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        save_total_limit=2,
        report_to='none',
        seed=42,
        fp16=torch.cuda.is_available(),  # ✅ FP16 only if CUDA available
    )

    # Initialize custom trainer with FOCAL LOSS
    trainer = BitNetTrainer(
        warmup_steps=1000,
        class_weight=class_weights,
        use_focal_loss=True,  # ✅ Using Focal Loss for better F1 Macro
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
    )

    # Train
    print("\n" + "="*60)
    print("STARTING OPTIMIZED TRAINING")
    print("="*60)
    results = trainer.train()

    # Evaluate
    print("\n" + "="*60)
    print("FINAL EVALUATION ON VALIDATION SET")
    print("="*60)
    eval_results = trainer.evaluate()
    print("\nValidation Metrics:")
    for key, value in eval_results.items():
        if isinstance(value, (int, float)):
            print(f"  {key}: {value:.4f}")

    # ✅ Proper way to save PyTorch model + HuggingFace tokenizer
    import os
    save_dir = './bitnet_polarization_model_optimized'
    os.makedirs(save_dir, exist_ok=True)

      # Save model state dict (PyTorch standard)
    torch.save({
          'model_state_dict': model.state_dict(),
          'model_config': {
              'model_name': 'bert-base-uncased',
              'num_labels': 2,
              'dropout_prob': 0.2
          }
      }, os.path.join(save_dir, 'pytorch_model.bin'))

      # Save tokenizer (HuggingFace - this method works for tokenizer!)
    tokenizer.save_pretrained(save_dir)


    return model, tokenizer, trainer, eval_results


In [ ]:
def predict_polarization(text, model, tokenizer, return_probabilities=True):
    """
    Make predictions on new text

    Args:
        text: Input text string
        model: Trained BitNet model
        tokenizer: BERT tokenizer
        return_probabilities: If True, return probabilities along with prediction

    Returns:
        prediction: 0 (Not Polarized) or 1 (Polarized)
        confidence: Probability of being polarized (if return_probabilities=True)
    """
    model.eval()

    with torch.no_grad():
        # Tokenize input
        inputs = tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=128
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Get model predictions
        outputs = model(**inputs)
        logits = outputs.logits

        # Convert to probabilities
        probs = torch.softmax(logits, dim=-1)
        pred = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][1].item()  # Probability of being polarized

    if return_probabilities:
        return pred, confidence
    return pred

In [ ]:
def create_submission(model, tokenizer, test_file='dev_eng.csv', output_file='dev_predictions.csv', threshold=0.48):
    """
    Create predictions for dev/test dataset with custom threshold

    Args:
        model: Trained model
        tokenizer: Tokenizer
        test_file: Path to test file
        output_file: Output CSV file
        threshold: Decision threshold (default 0.48)

    Returns:
        submission: DataFrame with predictions
    """
    print(f"\nCreating predictions from {test_file}...")
    print(f"Using threshold: {threshold}")

    # Load test data
    test = pd.read_csv(test_file)
    print(f"Test samples: {len(test)}")

    # Create dataset with dummy labels
    test_dataset = PolarizationDataset(
        test['text'].tolist(),
        [0] * len(test),  # Dummy labels
        tokenizer,
        max_length=128
    )

    # Create trainer for prediction
    trainer = Trainer(
        model=model,
        data_collator=DataCollatorWithPadding(tokenizer)
    )

    # Get predictions
    print("Generating predictions...")
    prediction_output = trainer.predict(test_dataset)
    raw_predictions = prediction_output.predictions

    # Convert to tensor and handle shape
    if not isinstance(raw_predictions, torch.Tensor):
        raw_predictions = torch.tensor(raw_predictions)

    # Ensure correct shape (num_samples, num_labels)
    if raw_predictions.dim() == 1:
        # If 1D, reshape to (num_samples, num_labels)
        raw_predictions = raw_predictions.reshape(-1, 2)
    elif raw_predictions.shape[1] != 2:
        # If shape is unexpected, try to fix it
        raw_predictions = raw_predictions.reshape(len(test), -1)[:, :2]

    # Apply softmax to get probabilities
    probs = F.softmax(raw_predictions, dim=1)
    pred_probs = probs[:, 1].numpy()  # Probability of polarized class

    # Apply threshold
    pred_labels = (pred_probs >= threshold).astype(int)

    # Create submission dataframe
    submission = pd.DataFrame({
        'id': test['id'],
        'text': test['text'],
        'predicted_polarization': pred_labels,
        'polarization_probability': pred_probs
    })

    # Save to CSV
    submission.to_csv(output_file, index=False)
    print(f"\nPredictions saved to: {output_file}")

    # Print statistics
    print(f"\nPrediction Statistics:")
    print(f"  Non-polarized (0): {(pred_labels == 0).sum()} ({(pred_labels == 0).sum() / len(pred_labels) * 100:.2f}%)")
    print(f"  Polarized (1): {(pred_labels == 1).sum()} ({(pred_labels == 1).sum() / len(pred_labels) * 100:.2f}%)")
    print(f"  Mean probability: {pred_probs.mean():.4f}")
    print(f"  Median probability: {np.median(pred_probs):.4f}")

    return submission


In [ ]:
def find_optimal_threshold(model, tokenizer, val_file='train_eng.csv'):
    """
    Find optimal threshold for F1 Macro maximization

    Uses the validation split from training data to find best threshold

    Args:
        model: Trained model
        tokenizer: Tokenizer
        val_file: Training file (will use 20% validation split)

    Returns:
        best_threshold: Optimal threshold
        best_f1_macro: Best F1 Macro achieved
        results_df: Full results DataFrame
    """
    print("\n" + "="*70)
    print("FINDING OPTIMAL THRESHOLD FOR F1 MACRO")
    print("="*70)

    # Load and split data same way as training
    full_data = pd.read_csv(val_file)
    _, val_data = train_test_split(
        full_data,
        test_size=0.2,
        stratify=full_data['polarization'],
        random_state=42
    )

    print(f"Validation samples: {len(val_data)}")

    # Create dataset
    val_dataset = PolarizationDataset(
        val_data['text'].tolist(),
        val_data['polarization'].tolist(),
        tokenizer,
        max_length=128
    )

    # Get predictions
    trainer = Trainer(
        model=model,
        data_collator=DataCollatorWithPadding(tokenizer)
    )

    print("Generating predictions...")
    predictions = trainer.predict(val_dataset)
    raw_predictions = predictions.predictions

    # Convert to probabilities
    if not isinstance(raw_predictions, torch.Tensor):
        raw_predictions = torch.tensor(raw_predictions)

    if raw_predictions.dim() == 1:
        raw_predictions = raw_predictions.reshape(-1, 2)

    probs = F.softmax(raw_predictions, dim=1)[:, 1].numpy()
    true_labels = val_data['polarization'].values

    # Test thresholds
    print("\nTesting thresholds from 0.30 to 0.70...")
    thresholds = np.arange(0.30, 0.71, 0.01)

    results = []
    for thresh in thresholds:
        pred_labels = (probs >= thresh).astype(int)

        f1_macro = f1_score(true_labels, pred_labels, average='macro')
        f1_binary = f1_score(true_labels, pred_labels, average='binary', zero_division=0)
        precision = (pred_labels[pred_labels == 1] == true_labels[pred_labels == 1]).sum() / max(pred_labels.sum(), 1)
        recall = (pred_labels[true_labels == 1] == 1).sum() / max((true_labels == 1).sum(), 1)
        accuracy = (pred_labels == true_labels).sum() / len(true_labels)

        results.append({
            'threshold': thresh,
            'f1_macro': f1_macro,
            'f1_binary': f1_binary,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy
        })

    # Create results DataFrame
    results_df = pd.DataFrame(results)

    # Find best threshold
    best_idx = results_df['f1_macro'].idxmax()
    best_result = results_df.loc[best_idx]

    # Print top 5 thresholds
    print("\nTop 5 thresholds by F1 Macro:")
    print("-"*70)
    print(f"{'Threshold':<12} {'F1 Macro':<12} {'F1 Binary':<12} {'Precision':<12} {'Recall':<12}")
    print("-"*70)

    top_5 = results_df.nlargest(5, 'f1_macro')
    for _, row in top_5.iterrows():
        print(f"{row['threshold']:.2f}         {row['f1_macro']:.4f}       "
              f"{row['f1_binary']:.4f}       {row['precision']:.4f}       {row['recall']:.4f}")

    print("\n" + "="*70)
    print("OPTIMAL THRESHOLD FOUND")
    print("="*70)
    print(f"Best Threshold: {best_result['threshold']:.2f}")
    print(f"F1 Macro: {best_result['f1_macro']:.4f}")
    print(f"F1 Binary: {best_result['f1_binary']:.4f}")
    print(f"Precision: {best_result['precision']:.4f}")
    print(f"Recall: {best_result['recall']:.4f}")
    print(f"Accuracy: {best_result['accuracy']:.4f}")

    # Save full results
    results_df.to_csv('threshold_optimization_results.csv', index=False)
    print("\nFull results saved to: threshold_optimization_results.csv")

    return best_result['threshold'], best_result['f1_macro'], results_df


In [ ]:
def test_inference_examples(model, tokenizer):
    """Test model on example texts"""

    test_examples = [
        "This politician is destroying our country with terrible policies!",
        "I believe we need better education and healthcare systems.",
        "Those people are all criminals and should be deported immediately!",
        "Research shows that renewable energy can reduce carbon emissions.",
        "They're trying to take away our rights and freedoms!",
        "The weather forecast predicts rain tomorrow afternoon.",
    ]

    print("\n" + "="*60)
    print("INFERENCE EXAMPLES")
    print("="*60)

    for i, text in enumerate(test_examples, 1):
        pred, confidence = predict_polarization(text, model, tokenizer)
        label = "Polarized" if pred == 1 else "Not Polarized"
        print(f"\n{i}. Text: {text}")
        print(f"   Prediction: {label}")
        print(f"   Confidence: {confidence:.3f}")

In [ ]:
if __name__ == "__main__":
    print("="*70)
    print("OPTIMIZED WORKFLOW FOR F1 MACRO > 0.80")
    print("="*70)

    # STEP 1: Train the model with optimized hyperparameters
    print("\n" + "="*70)
    print("STEP 1/4: TRAINING MODEL")
    print("="*70)

    model, tokenizer, trainer, train_results = train_polarization_detector()

    # STEP 2: Find optimal threshold
    print("\n" + "="*70)
    print("STEP 2/4: FINDING OPTIMAL THRESHOLD")
    print("="*70)

    optimal_threshold, best_f1_macro, threshold_results = find_optimal_threshold(
        model,
        tokenizer,
        val_file='train_eng.csv'
    )

    print(f"\n🎯 Optimal threshold: {optimal_threshold:.2f}")
    print(f"🎯 Expected F1 Macro: {best_f1_macro:.4f}")

    # STEP 3: Test on example texts
    print("\n" + "="*70)
    print("STEP 3/4: TESTING ON EXAMPLES")
    print("="*70)

    test_inference_examples(model, tokenizer)

    # STEP 4: Generate predictions for dev dataset with optimal threshold
    print("\n" + "="*70)
    print("STEP 4/4: GENERATING DEV PREDICTIONS")
    print("="*70)

    submission = create_submission(
        model,
        tokenizer,
        test_file='dev_eng.csv',
        output_file='dev_predictions_optimized.csv',
        threshold=optimal_threshold  # ✅ Using optimal threshold
    )

    # Final summary
    print("\n" + "="*70)
    print("TRAINING AND PREDICTION COMPLETE!")
    print("="*70)

    print("\n📊 FINAL SUMMARY:")
    print(f"   Model: BitNet with Focal Loss (α=0.65, γ=2.0)")
    print(f"   Training epochs: 10")
    print(f"   Dropout: 0.2")
    print(f"   Learning rate: 2e-5")
    print(f"   Weight decay: 0.02")
    print(f"   Optimal threshold: {optimal_threshold:.2f}")
    print(f"   Validation F1 Macro: {best_f1_macro:.4f}")

    print("\n📁 OUTPUT FILES:")
    print("   ✅ Model: ./bitnet_polarization_model_optimized/")
    print("   ✅ Predictions: dev_predictions_optimized.csv")
    print("   ✅ Threshold analysis: threshold_optimization_results.csv")

    print("\n💡 PERFORMANCE ANALYSIS:")
    if best_f1_macro >= 0.80:
        print(f"   ✅ SUCCESS! F1 Macro = {best_f1_macro:.4f} >= 0.80")
        print("   🎉 Target achieved!")
    elif best_f1_macro >= 0.78:
        print(f"   ⚡ CLOSE! F1 Macro = {best_f1_macro:.4f}")
        print("   Try: Increase epochs to 12-15 or try RoBERTa-base")
    else:
        print(f"   ⚠️  Current F1 Macro = {best_f1_macro:.4f}")
        print("   Suggestions:")
        print("   - Try RoBERTa: model_name = 'roberta-base'")
        print("   - Train ensemble of 3-5 models")
        print("   - Tune Focal Loss: alpha=[0.6, 0.65, 0.7], gamma=[1.5, 2.0, 2.5]")

    print("\n" + "="*70)


OPTIMIZED WORKFLOW FOR F1 MACRO > 0.80

STEP 1/4: TRAINING MODEL
SUBTASK 1: BINARY POLARIZATION DETECTION
1-Bit LLM with BitLinear Quantization - OPTIMIZED

Loading data...
Training samples: 2140
Validation samples: 536

Training label distribution:
polarization
0    1339
1     801
Name: count, dtype: int64

Validation label distribution:
polarization
0    335
1    201
Name: count, dtype: int64

Initializing tokenizer: bert-base-uncased
Creating datasets...

Initializing BitNet model...
Loading BERT model: bert-base-uncased
Model initialized with 109,780,610 parameters
Total parameters: 109,780,610
Trainable parameters: 109,780,610
Class weights: tensor([0.8200, 1.3000])
Lambda warmup enabled: 0 -> 1 over 1000 steps
Using Focal Loss (alpha=0.65, gamma=2.0)

STARTING OPTIMIZED TRAINING


Step,Training Loss,Validation Loss,F1 Macro,F1 Binary,Accuracy
100,0.183700,0.108141,0.615971,0.527845,0.636194
200,0.162500,0.113363,0.698824,0.602740,0.729478
300,0.130900,0.107354,0.733155,0.694323,0.738806
400,0.101000,0.106136,0.734485,0.650273,0.761194
500,0.069100,0.135308,0.740325,0.670077,0.759328
600,0.043300,0.148034,0.762899,0.696104,0.781716
700,0.030200,0.163519,0.776862,0.728571,0.787313
